# This notebook referes to unit 4 lesson 4.2

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
from collections import Counter
import nltk

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_validate, cross_val_score
from sklearn import svm

nlp = spacy.load('en')

### Challenge 0:
- try SVM 
- feature engineer 
- get score >= 90

In [2]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text
    
# Load and clean the data.
persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

# The Chapter indicator is idiosyncratic
persuasion = re.sub(r'Chapter \d+', '', persuasion)
alice = re.sub(r'CHAPTER .*', '', alice)
    
alice = text_cleaner(alice[:int(len(alice)/10)])
persuasion = text_cleaner(persuasion[:int(len(persuasion)/10)])

In [3]:
# Parse the cleaned novels. This can take a bit.
nlp = spacy.load('en')
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)

In [4]:
# Group into sentences.
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]

# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(alice_sents + persuasion_sents)
sentences.head()

,0,1
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !)",Carroll
4,"(Oh, dear, !)",Carroll


In [5]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 50 == 0:
            print("Processing row {}".format(i))
            
    return df

# Set up the bags.
alicewords = bag_of_words(alice_doc)
persuasionwords = bag_of_words(persuasion_doc)

# Combine bags to create a set of unique words.
common_words = set(alicewords + persuasionwords)

In [6]:
# Create our data frame with features. This can take a while to run.
word_counts = bow_features(sentences, common_words)
word_counts.head()

Processing row 0
Processing row 50
Processing row 100
Processing row 150
Processing row 200
Processing row 250
Processing row 300
Processing row 350
Processing row 400


,commander,powerful,rapid,Baldwin,rent,creep,pop,sure,principal,unsolicitous,...,fountain,known,finish,genteel,fool,spirit,expedition,intention,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll


In [7]:
# find the sentence length
length = []
for i in range(len(sentences)):
    length.append(len(sentences[0].iloc[i]))

# add the length as a feature to df at position 1 before the end
word_counts.insert(len(word_counts.columns) -1, 'sentence_length', length)
word_counts.head()

,commander,powerful,rapid,Baldwin,rent,creep,pop,sure,principal,unsolicitous,...,known,finish,genteel,fool,spirit,expedition,intention,text_sentence,sentence_length,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",67,Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",63,Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",30,Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(Oh, dear, !)",3,Carroll
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(Oh, dear, !)",3,Carroll


In [8]:
# define lists
alpha_list, adj_list, adv_list, aux_list, conj_list = [], [], [], [], []
det_list, noun_list, num_list, part_list, pron_list, verb_list = [], [], [], [], [], []

# count each the values from each and add to correct list
for i in range(len(sentences)):
    # get eacj sentence
    doc = sentences[0].iloc[i]
    
    # define variables
    alpha_cnt, adj_cnt, adv_cnt, aux_cnt, conj_cnt, det_cnt = 0, 0, 0, 0, 0, 0
    noun_cnt, num_cnt, part_cnt, pron_cnt, verb_cnt = 0, 0, 0, 0, 0
    
    # check each work in the sentence
    for token in doc:
        if token.is_alpha:
            alpha_cnt += 1
        if token.pos_ == 'ADJ':
            adj_cnt += 1
        if token.pos_ == 'ADV':
            adv_cnt += 1
        if token.pos_ == 'AUX':
            aux_cnt += 1
        if token.pos_ == 'CONJ':
            conj_cnt += 1
        if token.pos_ == 'DET':
            det_cnt += 1
        if token.pos_ == 'NOUN':
            noun_cnt += 1
        if token.pos_ == 'NUM':
            num_cnt += 1
        if token.pos_ == 'PART':
            part_cnt += 1
        if token.pos_ == 'PRON':
            pron_cnt += 1
        if token.pos_ == 'VERB':
            verb_cnt += 1
            
    # add values to lists        
    alpha_list.append(alpha_cnt)
    adj_list.append(adj_cnt)
    adv_list.append(adv_cnt)
    aux_list.append(aux_cnt)
    conj_list.append(conj_cnt)
    det_list.append(det_cnt)
    noun_list.append(noun_cnt)
    num_list.append(num_cnt)
    part_list.append(part_cnt)
    pron_list.append(pron_cnt)
    verb_list.append(verb_cnt)

In [9]:
# add new features to df
word_counts.insert(len(word_counts.columns) -3, 'alpha_cnt', alpha_list)
word_counts.insert(len(word_counts.columns) -3, 'adj_cnt', adj_list)
word_counts.insert(len(word_counts.columns) -3, 'adv_cnt', adv_list)
word_counts.insert(len(word_counts.columns) -3, 'aux_cnt', aux_list)
word_counts.insert(len(word_counts.columns) -3, 'conj_cnt', conj_list)
word_counts.insert(len(word_counts.columns) -3, 'det_cnt', det_list)
word_counts.insert(len(word_counts.columns) -3, 'noun_cnt', noun_list)
word_counts.insert(len(word_counts.columns) -3, 'num_cnt', num_list)
word_counts.insert(len(word_counts.columns) -3, 'part_cnt', part_list)
word_counts.insert(len(word_counts.columns) -3, 'pron_cnt', pron_list)
word_counts.insert(len(word_counts.columns) -3, 'verb_cnt', verb_list)

word_counts.head()

,commander,powerful,rapid,Baldwin,rent,creep,pop,sure,principal,unsolicitous,...,conj_cnt,det_cnt,noun_cnt,num_cnt,part_cnt,pron_cnt,verb_cnt,text_sentence,sentence_length,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,7,11,0,2,4,13,"(Alice, was, beginning, to, get, very, tired, ...",67,Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,7,8,0,1,4,11,"(So, she, was, considering, in, her, own, mind...",63,Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,3,2,0,1,2,5,"(There, was, nothing, so, VERY, remarkable, in...",30,Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(Oh, dear, !)",3,Carroll
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(Oh, dear, !)",3,Carroll


In [10]:
y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.4,random_state=0)

lr = LogisticRegression(penalty='l2', solver='lbfgs', max_iter=200) 
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

(266, 1624) (266,)
Training set score: 0.981203007518797

Test set score: 0.8707865168539326


In [11]:
# SVM
clf = svm.SVC(gamma='scale', kernel='linear', probability=True)
clf.fit(X_train, y_train)

print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))

Training set score: 0.9774436090225563

Test set score: 0.8651685393258427


In [12]:
# Stochastic Gradient Descent
from sklearn.linear_model import SGDClassifier
sgd = SGDClassifier(penalty='elasticnet', loss='log')
sgd.fit(X_train, y_train)
print('Training set score:', sgd.score(X_train, y_train))
print('\nTest set score:', sgd.score(X_test, y_test))

Training set score: 0.9172932330827067

Test set score: 0.848314606741573


In [13]:
# cross valuate for SVM
cv_results = cross_validate(clf, X, y, cv=5)
cv_results

{'fit_time': array([1.19280839, 1.12901998, 1.2097621 , 1.13896251, 1.15816426]),
 'score_time': array([0.036901  , 0.03386927, 0.03291416, 0.04287744, 0.02692771]),
 'test_score': array([0.80898876, 0.83146067, 0.87640449, 0.80898876, 0.73863636])}

In [14]:
# Gaussian
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()
nb.fit(X_train, y_train)

cv_results = cross_validate(nb, X, y, cv=10)
cv_results['test_score']

array([0.88888889, 0.84444444, 0.88888889, 0.82222222, 0.91111111,
       0.84090909, 0.81818182, 0.65909091, 0.79545455, 0.86046512])

## Challenge 1:
copmare Alice vs other book

In [15]:
# print list of options
print(gutenberg.fileids())

['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt', 'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt', 'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt', 'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt', 'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'shakespeare-macbeth.txt', 'whitman-leaves.txt']


In [34]:
# load data
moby = gutenberg.raw('melville-moby_dick.txt')

# print first few char
moby[:105]

'[Moby Dick by Herman Melville 1851]\r\n\r\n\r\nETYMOLOGY.\r\n\r\n(Supplied by a Late Consumptive Usher to a Grammar'

In [35]:
# clean keep 1/60 of the length
moby = text_cleaner(moby[:int(len(moby)/60)])
moby[:200]

'ETYMOLOGY. (Supplied by a Late Consumptive Usher to a Grammar School) The pale Usher threadbare in coat, heart, body, and brain; I see him now. He was ever dusting his old lexicons and grammars, with '

In [36]:
# Parse the cleaned novel
moby_doc = nlp(moby)
len(moby_doc)

4344

In [37]:
# NEW df with Alice and Moby Dick.
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
moby_sents = [[sent, "Melville"] for sent in moby_doc.sents]

# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(alice_sents + moby_sents)
sentences.tail()

,0,1
541,"("")",Melville
542,"(It, was, not, till, the, boats, returned, fro...",Melville
543,"("", NEWSPAPER, ACCOUNT, OF, THE, TAKING, AND, ...",Melville
544,"("")",Melville
545,"(It, is, generally, well, known, that, out, of...",Melville


In [38]:
# Set up the bags.
alicewords = bag_of_words(alice_doc)
mobywords = bag_of_words(moby_doc)

# Combine bags to create a set of unique words.
common_words = set(alicewords + mobywords)

word_counts = bow_features(sentences, common_words)
word_counts.head()

Processing row 0
Processing row 50
Processing row 100
Processing row 150
Processing row 200
Processing row 250
Processing row 300
Processing row 350
Processing row 400
Processing row 450
Processing row 500


,soever,indirectly,creep,pop,COMMODORE,sure,bone,ALICE,PROFANE,little,...,milk,shine,restless,vent,eat,fountain,UNO,finish,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll


In [39]:
# find the sentence length
length = []
for i in range(len(sentences)):
    length.append(len(sentences[0].iloc[i]))

# add the length as a feature to df at position 1 before the end
word_counts.insert(len(word_counts.columns) -1, 'sentence_length', length)
word_counts.tail()

,soever,indirectly,creep,pop,COMMODORE,sure,bone,ALICE,PROFANE,little,...,shine,restless,vent,eat,fountain,UNO,finish,text_sentence,sentence_length,text_source
541,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"("")",1,Melville
542,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(It, was, not, till, the, boats, returned, fro...",31,Melville
543,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"("", NEWSPAPER, ACCOUNT, OF, THE, TAKING, AND, ...",15,Melville
544,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"("")",1,Melville
545,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(It, is, generally, well, known, that, out, of...",26,Melville


In [40]:
# define lists
alpha_list, adj_list, adv_list, aux_list, conj_list = [], [], [], [], []
det_list, noun_list, num_list, part_list, pron_list, verb_list = [], [], [], [], [], []

# count each the values from each and add to correct list
for i in range(len(sentences)):
    # get eacj sentence
    doc = sentences[0].iloc[i]
    
    # define variables
    alpha_cnt, adj_cnt, adv_cnt, aux_cnt, conj_cnt, det_cnt = 0, 0, 0, 0, 0, 0
    noun_cnt, num_cnt, part_cnt, pron_cnt, verb_cnt = 0, 0, 0, 0, 0
    
    # check each work in the sentence
    for token in doc:
        if token.is_alpha:
            alpha_cnt += 1
        if token.pos_ == 'ADJ':
            adj_cnt += 1
        if token.pos_ == 'ADV':
            adv_cnt += 1
        if token.pos_ == 'AUX':
            aux_cnt += 1
        if token.pos_ == 'CONJ':
            conj_cnt += 1
        if token.pos_ == 'DET':
            det_cnt += 1
        if token.pos_ == 'NOUN':
            noun_cnt += 1
        if token.pos_ == 'NUM':
            num_cnt += 1
        if token.pos_ == 'PART':
            part_cnt += 1
        if token.pos_ == 'PRON':
            pron_cnt += 1
        if token.pos_ == 'VERB':
            verb_cnt += 1
            
    # add values to lists        
    alpha_list.append(alpha_cnt)
    adj_list.append(adj_cnt)
    adv_list.append(adv_cnt)
    aux_list.append(aux_cnt)
    conj_list.append(conj_cnt)
    det_list.append(det_cnt)
    noun_list.append(noun_cnt)
    num_list.append(num_cnt)
    part_list.append(part_cnt)
    pron_list.append(pron_cnt)
    verb_list.append(verb_cnt)

In [41]:
# add new features to df
word_counts.insert(len(word_counts.columns) -3, 'alpha_cnt', alpha_list)
word_counts.insert(len(word_counts.columns) -3, 'adj_cnt', adj_list)
word_counts.insert(len(word_counts.columns) -3, 'adv_cnt', adv_list)
word_counts.insert(len(word_counts.columns) -3, 'aux_cnt', aux_list)
word_counts.insert(len(word_counts.columns) -3, 'conj_cnt', conj_list)
word_counts.insert(len(word_counts.columns) -3, 'det_cnt', det_list)
word_counts.insert(len(word_counts.columns) -3, 'noun_cnt', noun_list)
word_counts.insert(len(word_counts.columns) -3, 'num_cnt', num_list)
word_counts.insert(len(word_counts.columns) -3, 'part_cnt', part_list)
word_counts.insert(len(word_counts.columns) -3, 'pron_cnt', pron_list)
word_counts.insert(len(word_counts.columns) -3, 'verb_cnt', verb_list)

word_counts.head()

,soever,indirectly,creep,pop,COMMODORE,sure,bone,ALICE,PROFANE,little,...,conj_cnt,det_cnt,noun_cnt,num_cnt,part_cnt,pron_cnt,verb_cnt,text_sentence,sentence_length,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,7,11,0,2,4,13,"(Alice, was, beginning, to, get, very, tired, ...",67,Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,7,8,0,1,4,11,"(So, she, was, considering, in, her, own, mind...",63,Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,3,2,0,1,2,5,"(There, was, nothing, so, VERY, remarkable, in...",30,Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(Oh, dear, !)",3,Carroll
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(Oh, dear, !)",3,Carroll


In [42]:
y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)

lr = LogisticRegression(penalty='l2', solver='lbfgs', max_iter=200) 
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

(327, 1450) (327,)
Training set score: 0.9571865443425076

Test set score: 0.9315068493150684


In [43]:
pred = lr.predict(X_test)
pd.crosstab(y_test, pred)

col_0,Carroll,Melville
text_source,,
Carroll,33,10
Melville,5,171
